In [ ]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 19.9 MB/s eta 0:00:00


In [ ]:
from z3 import *
import numpy as np
import time

In [ ]:
def maximum(x):
    m = x[0]
    for v in x[1:]:
        m = If(v > m, v, m)
    return m

def run_model_on_instance(file):
    with open(file) as f:
      m = int(next(f)) # couriers
      n = int(next(f)) # items
      l = [int(e) for e in next(f).split()] # capacities
      s = [int(e) for e in next(f).split()] # max size of packages a courier can carry
      D = np.genfromtxt(f, dtype=int).tolist() # Distances
    return m, n, l, s, D

In [55]:
def SMT(m, n, l, s, D):
  COURIERS = range(m)
  ITEMS = range(n)

  ####################################### DECISION VARIABLES #######################################

  # main decision variable: x[i,j] = k mean that the i-th courier is in j at time k
  X = [[Int(f'X_{i}_{j}') for j in ITEMS]for i in COURIERS]

  # variable for distance calculation
  dist = [Int(f'dist_{i}') for i in COURIERS]

  # variable for loads calculation
  load = [Int(f'load_{i}') for i in COURIERS]

  # the number of items that are delivered by each courier
  count = [Int(f'count_{i}') for i in COURIERS]

  solver = Solver()

  ####################################### CONSTRAINTS #######################################

  # Assignment Constraint
  for j in ITEMS:
    solver.add(Sum([If(X[i][j] > 0, 1, 0) for i in COURIERS]) == 1)

  for i in COURIERS:
    solver.add(Sum([If(X[i][j] > 0, 1, 0) for j in ITEMS]) >= 1)

  # Load Constraint
  for i in COURIERS:
    solver.add(Sum([If(X[i][j] > 0, s[j], 0) for j in ITEMS]) <= l[i])

  for i in COURIERS:  # For each courier
    # Calculate the number of assigned items (`c`) for courier `i`
    c = Sum([If(X[i][j] > 0, 1, 0) for j in ITEMS])
    solver.add(count[i] == c)
    # Distance from origin to the first item in the route
    dist_start = Sum([If(X[i][j] == 1, D[n][j], 0) for j in ITEMS])

    # Distance between consecutive items in the route
    dist_consecutive = Sum([
        If(And(X[i][j1] > 0, X[i][j2] > 0, X[i][j2] - X[i][j1] == 1), D[j1][j2], 0)
        for j1 in ITEMS for j2 in ITEMS
    ])

    # Distance from the last item in the route back to the origin
    dist_end = Sum([If(X[i][j] == c, D[j][n], 0) for j in ITEMS])

    # Total distance expression for courier `i`
    dist_expr = dist_start + dist_consecutive + dist_end

    # Add the constraint for the total distance of courier `i`
    solver.add(dist[i] == dist_expr)

  for i in COURIERS:
    for j in ITEMS:
      solver.add(And(X[i][j] >= 0, X[i][j] <= count[i]))

  for i in COURIERS:
    items = [If(X[i][j] > 0, X[i][j], -j) for j in ITEMS]
    solver.add(Distinct(items))

  ####################################### OBJECTIVE FUNCTION ######################################

  obj = Int('obj')
  solver.add(obj == maximum([dist[i] for i in COURIERS]))

  ######################################## SEARCH STRATEGY ########################################

  lower_bound = max([D[n][j] + D[j][n] for j in ITEMS])
  solver.add(obj >= lower_bound)

  ############################################# SOLVE #############################################

  if solver.check() != sat:
    print ("failed to solve")

  final_value = 0
  while solver.check() == sat:
    model = solver.model()
    result_X = [ [ model.evaluate(X[i][j]) for j in ITEMS ]
            for i in COURIERS ]
    result_dist = [model.evaluate(dist[i]) for i in COURIERS]
    result_objective = model.evaluate(obj)

    for i in COURIERS:
      print_matrix(result_X[i])

    print()
    print_matrix(result_dist)
    print(f"Intermediate objective value: {result_objective}\n")
    final_value = result_objective
    solver.add(obj < result_objective)

  print(f"\n\nFinal objective: {final_value}")
  # final_time = time.time() - start_time
  # print(f"Finished in: {final_time:3.3} seconds\n")



In [60]:
SMT(*run_model_on_instance("inst10.dat"))

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

[206, 114, 152, 120, 146, 216, 158, 244, 198, 228]
Intermediate objective value: 244



Final objective: 244
